In [1]:
from data import CustomDataset
from transformers import GPT2LMHeadModel
import torch
from torch.optim import AdamW
from tqdm import tqdm
from torch.utils.data import DataLoader, random_split


d:\app\Python\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
d:\app\Python\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvisi

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_obj = CustomDataset('./dataset/dataset.csv')
dataset = dataset_obj.generate_dataset()


Map:   0%|          | 0/104 [00:00<?, ? examples/s]

In [4]:
model = GPT2LMHeadModel.from_pretrained('./Model/Original')
model.resize_token_embeddings(len(dataset_obj.tokenizer))
model.to(device)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [5]:
train_size = int(0.9 * len(dataset))  # 90% cho train
val_size = len(dataset) - train_size  # 10% cho validation

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(val_dataset, batch_size=8)

# Hàm đánh giá
def evaluate_model(model, validation_dataloader):
    model.eval()  # Đặt mô hình ở chế độ đánh giá
    total_loss = 0

    with torch.no_grad():  # Tắt gradient để tăng tốc độ và tiết kiệm bộ nhớ
        for batch in tqdm(validation_dataloader, desc="Evaluating", leave=False):
            inputs = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = inputs.clone()

            # Tính loss
            outputs = model(inputs, labels=labels, attention_mask=attention_mask)
            loss = outputs.loss
            total_loss += loss.item()
    
    avg_loss = total_loss / len(validation_dataloader)
    return avg_loss


In [6]:

optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 1

for epoch in range(num_epochs):
    model.train()  # Đặt mô hình ở chế độ huấn luyện
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", leave=False):
        # Lấy inputs và labels
        inputs = batch['input_ids'].to(device)
        
        attention_mask = batch['attention_mask'].to(device)
        labels = inputs.clone()

        outputs = model(inputs, labels=labels, attention_mask=attention_mask)
        
        optimizer.zero_grad()  # Reset gradients
        loss = outputs.loss
        loss.backward()

        optimizer.step()  # Cập nhật tham số

        total_loss += loss.item()

    # Đánh giá sau mỗi epoch
    val_loss = evaluate_model(model, validation_dataloader)
    print(f"Epoch {epoch + 1}, Train Loss: {total_loss / len(train_dataloader)}, Val Loss: {val_loss}")


KeyboardInterrupt: 

In [8]:
user = 'Xin chào bạn'
input_text = f'<|im_start|>user<|im_sep|>{user}<|im_end|><|im_start|>assistant<|im_sep|>'
inputs = dataset_obj.tokenizer(input_text, return_tensors="pt").to(device)

model.eval()
outputs = model.generate(
    inputs['input_ids'],
    pad_token_id=dataset_obj.tokenizer.pad_token_id,
    attention_mask = inputs['attention_mask'],
    do_sample=True,
    max_length=256,
    min_length=10,
    top_k=40,
    num_beams=5,
    early_stopping=True,
    no_repeat_ngram_size=2,
    num_return_sequences=1
)

# Giải mã kết quả
generated_texts = dataset_obj.tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_texts)


<|im_start|> user <|im_sep|> Xin chào bạn <|im_end|> <|im_start|> assistant <|im_sep|> "},{},{}) })();
(GDVN) - Bộ GD&ĐT vừa công bố dự thảo Quy chế thi THPT quốc gia và xét công nhận tốt nghiệp THPT năm 2017. Theo đó, quy chế này sẽ có hiệu lực kể từ ngày 1/7/2017.
Bộ Giáo dục và Đào tạo vừa ban hành Thông tư sửa đổi, bổ sung một số điều của các thông tư quy định về thi, kiểm tra, xử lý kết quả học tập của học sinh, sinh viên (HSSV).
Thông tư này cũng nêu rõ, các cơ sở giáo dục
